In [1]:
import numpy
from Solver import *
from copy import deepcopy
from ImprovementAlgorithm import *
from Neighborhood import *
import time
import random

---
### Input Data
---

In [2]:
class DataNode:
    def __init__(self, Id, X, Y, Score):
        self.__nodeId = Id
        self.__X = X
        self.__Y = Y
        self.Times = []
        self.__score = Score
        
    def __str__(self):
        return f"Node {self.__nodeId} with Score {self.__score} at ({self.__X}, {self.__Y})"
    
    @property
    def Id(self):
        return self.__nodeId
    @property
    def X(self):
        return self.__X
    @property
    def Y(self):
        return self.__Y
    @property
    def Score(self):
        return self.__score

class InputData:
    def __init__(self, path):

        self.__Path = path
        self.__NodeCount = -1
        self.__TimeLimit = -1
        self.DataLoad()
        
    def DataLoad(self):

        with open(self.__Path, "r") as inputFile:
            inputData = json.load(inputFile)
        
        self.__NodeCount = inputData["NodeCount"]
        self.__TimeLimit = inputData["TimeLimit"]
        self.InputNodes = list()
        #Einlesen und Abspeichern der Nodes aus dem Input File
        for node in inputData["Nodes"]:
            self.InputNodes.append(DataNode(node["Id"], node["X"], node["Y"], node["Score"]))
        #Berechnung der Entfernung für den jeweiligen Knoten
        for i in range(len(inputData["Nodes"])):
            for j in range(len(inputData["Nodes"])):
                x1, y1 = (inputData["Nodes"][i]["X"], inputData["Nodes"][i]["Y"])
                x2, y2 = (inputData["Nodes"][j]["X"], inputData["Nodes"][j]["Y"])
                time = ((x2 - x1) ** 2 + (y2 - y1) ** 2) ** 0.5
                self.InputNodes[i].Times.append(time)
    @property
    def NodeCount(self):
        return self.__NodeCount
    @property
    def TimeLimit(self):
        return self.__TimeLimit
    @property
    def Path(self):
        return self.__Path

---
### Output Data
---

In [3]:
class OutputNode(DataNode):
    def __init__(self, dataNode):
        super().__init__(dataNode.Id, dataNode.X, dataNode.Y, dataNode.Score)
        self.Position = -1
        self.Times = dataNode.Times
        
class Solution:
    def __init__(self, solutionNodes, sequence):
        self.OutputNodes = {}
        # Sobald eine Node sich in einer Tour befindet, wird diese zur Output Node
        for node in solutionNodes:
            self.OutputNodes[node.Id] = OutputNode(node)
        self.Sequence = sequence
        self.TotalScore = -1
        self.TotalTime = -1

    def __str__(self):
        #da Startknoten nicht in Lösungen hinzugefügt wird, muss er Extra erwähnt werden
        return f"Sequence {[1]+ self.Sequence +[1]} collects {self.TotalScore} score points in {self.TotalTime} time units"

class SolutionPool:
    def __init__(self):
        self.Solutions = []

    def AddSolution(self, newSolution):
        self.Solutions.append(newSolution)

    def GetHighestTotalScoreSolution(self):
        #complex sort: sorts are stable -> multiple records have the same key, their original order is preserved
        # sort solutions ascending by TotalTime
        self.Solutions.sort(key = lambda solution: solution.TotalTime, reverse=False)
        # sort solutions descending TotalScore
        self.Solutions.sort(key = lambda solution: solution.TotalScore, reverse=True)

        return self.Solutions[0]

---
### Evaluation Logic
---

In [4]:
class EvaluationLogic:
    def __init__(self, inputData):
        self.InputData = inputData

    def __init__(self, inputData):
        self.InputData = inputData
    
    def DefineTotalValues(self, currentSolution):
        total_score = 0
        total_time = 0

        # einfache Summe aller Scores in der Lösung
        for key in currentSolution.OutputNodes.keys():
            total_score += currentSolution.OutputNodes[key].Score
        
        #Entfernung Depot-"erster Knoten"
        total_time += currentSolution.OutputNodes[currentSolution.Sequence[0]].Times[0]
        #Entfernung Depot-"letzter Knoten"
        total_time += currentSolution.OutputNodes[currentSolution.Sequence[-1]].Times[0]
        # if total_time >= self.InputData.TimeLimit:
        #     raise Exception("Das Time Limit wurde bereits mit dem ersten Knoten überschritten.")
        #Berechnung der Entfernungen zwischen "erstem Knoten" und "letzen Knoten"
        for i in range(len(currentSolution.Sequence)-1):
            total_time += currentSolution.OutputNodes[currentSolution.Sequence[i]].Times[currentSolution.Sequence[i+1]-1]

        currentSolution.TotalScore = total_score
        currentSolution.TotalTime = total_time

---
### Constructive Heuristic
---

In [5]:
class ConstructiveHeuristics:
    def __init__(self, evaluationLogic, solutionPool):
        self.EvaluationLogic = evaluationLogic
        self.SolutionPool = solutionPool

    # def HighestScoreFirst(self, nodeList, evaluationLogic): #der Reihe nach wird höchster Score hinzugefügt
    #     #Triviallösung: alle passen in Route checken
    #     #Triviallösung: keine einzige passt in Route checken -> in EvalLogic berücksichtigt
    #     sortedNodes = sorted(nodeList, key = lambda x: x.Score, reverse = True)
    #     currentSolution = Solution([],[]) #Initialisierung der "leeren" Lösung
        
    #     for node in sortedNodes:
    #         #Hinzufügen des höchstbewertetsten Knoten
    #         currentSolution.OutputNodes[node.Id] = node
    #         currentSolution.Sequence.append(node.Id)
    #         #Bewertung der Lösung
    #         evaluationLogic.DefineTotalValues(currentSolution)
    #         #Ist generierte Lösung unzulässig, wird der letzte Schritt revidiert und die Konstruktion abgebrochen
    #         if currentSolution.TotalTime > evaluationLogic.InputData.TimeLimit:
    #             del currentSolution.OutputNodes[node.Id]
    #             currentSolution.Sequence.pop()
    #             evaluationLogic.DefineTotalValues(currentSolution)
    #             break
    #     return currentSolution

    #der Reihe nach wird höchster Score hinzugefügt
    def HighestScoreFirst(self, nodeList, evaluationLogic):

        sortedNodes = sorted(nodeList, key = lambda x: x.Score, reverse = True)
        #Knoten 0 entfernen
        sortedNodes = sortedNodes[:-1]
        #Initialisierung der "leeren" Lösung
        currentSolution = Solution([],[])

        while sortedNodes:
            nextNode = sortedNodes.pop(0)
            #Hinzufügen des höchstbewertetsten Knoten
            currentSolution.OutputNodes[nextNode.Id] = nextNode
            currentSolution.Sequence.append(nextNode.Id)    
            evaluationLogic.DefineTotalValues(currentSolution)
            #wenn der Knoten "zu viel" war, wird er wieder rausgenommen
            if currentSolution.TotalTime > evaluationLogic.InputData.TimeLimit:
                del currentSolution.OutputNodes[nextNode.Id]
                currentSolution.Sequence.pop(-1)
                
        evaluationLogic.DefineTotalValues(currentSolution)
        return currentSolution
    
    def NearestNeighborFirst(self, nodeList): #der Reihe nach wird der am nächsten liegende Knoten hinzugefügt
        currentSolution = Solution([],[]) #Initialisierung der "leeren" Lösung
        #sortieren nach Entfernung vom Depot
        sortedNodes = sorted(nodeList, key = lambda x: x.Times[0], reverse = False)
        del sortedNodes[0] #Depot entfernen
        #add nearest Node
        currentSolution.OutputNodes[sortedNodes[0].Id] = sortedNodes[0]
        currentSolution.Sequence.append(sortedNodes[0].Id)
        self.EvaluationLogic.DefineTotalValues(currentSolution)
        #nach dem Hinzufügen wird Knoten aus Menge entfernt 
        del sortedNodes[0]

        #Schrittweises hinzufügen aller Knoten bis die Restriktion erreicht ist (oder alle Knoten erreicht)
        while sortedNodes:
            sortedNodes = sorted(sortedNodes, key=lambda x: x.Times[currentSolution.Sequence[-1] - 1], reverse=False)
            currentSolution.OutputNodes[sortedNodes[0].Id] = sortedNodes[0] #add nearest Node
            currentSolution.Sequence.append(sortedNodes[0].Id)
            del sortedNodes[0]
            self.EvaluationLogic.DefineTotalValues(currentSolution)
        #Sobald TimeLimit überschritten wird abgebrochen und der letzte Knoten wieder entfernt
            if currentSolution.TotalTime > self.EvaluationLogic.InputData.TimeLimit:
                break
        del currentSolution.OutputNodes[currentSolution.Sequence[-1]]
        currentSolution.Sequence.pop()
        self.EvaluationLogic.DefineTotalValues(currentSolution)

        return currentSolution
    
    def Run(self, inputData, solutionMethod): #analog zur Vorlesung
        print('Generating an initial solution according to ' + solutionMethod + '.')

        solution = None

        if solutionMethod == 'HSF':
            solution = self.HighestScoreFirst(inputData.InputNodes, self.EvaluationLogic)
        elif solutionMethod == 'NNF':
            solution = self.NearestNeighborFirst(inputData.InputNodes)
        else:
            print('Unkown constructive solution method: ' + solutionMethod + '.')
        
        self.SolutionPool.AddSolution(solution)

---
### Neighborhood
---

In [6]:
class BaseNeighborhood:
    def __init__(self, inputData, initialSequence, evaluationLogic, solutionPool):
        self.InputData = inputData
        self.Sequence = initialSequence
        self.EvaluationLogic = evaluationLogic
        self.SolutionPool = solutionPool

        self.Moves = [] #Lösungen
        self.MoveSolutions = []#bewertete Lösungen

        self.Type = 'None'

    def DiscoverMoves(self):
        raise Exception('DiscoverMoves() is not implemented for the abstract BaseNeighborhood class.')

    def EvaluateMoves(self, evaluationStrategy):
        if evaluationStrategy == 'BestImprovement':
            self.EvaluateMovesBestImprovement()
        elif evaluationStrategy == 'FirstImprovement':
            self.EvaluateMovesFirstImprovement()
        else:
            raise Exception(f'Evaluation strategy {evaluationStrategy} not implemented.')
        
    def EvaluateMove(self, move):
        
        solutionNodes = []
        for node in self.InputData.InputNodes:
            if node.Id in move.Sequence:
                solutionNodes.append(node)
        #Erstellen einer Lösung aus dem move
        moveSolution = Solution(solutionNodes, move.Sequence)
        
        self.EvaluationLogic.DefineTotalValues(moveSolution)

        return moveSolution
    
    """ Evaluate all moves. """
    def EvaluateMovesBestImprovement(self):
        
        for move in self.Moves:
            moveSolution = self.EvaluateMove(move)
            #nur hinzufügen, wenn auch feasible!
            if moveSolution.TotalTime <= self.InputData.TimeLimit:
                self.MoveSolutions.append(moveSolution)

    """ Evaluate all moves until the first one is found that improves the best solution found so far. """
    def EvaluateMovesFirstImprovement(self):
        bestObjective = self.SolutionPool.GetHighestTotalScoreSolution().TotalScore #hier

        for move in self.Moves:
            moveSolution = self.EvaluateMove(move)
            #nur hinzufügen, wenn auch feasible!
            if moveSolution.TotalTime <= self.InputData.TimeLimit:
                self.MoveSolutions.append(moveSolution)
            if moveSolution.TotalScore > bestObjective:
                # abort neighborhood evaluation because an improvement has been found
                return
            
    def MakeBestMove(self):
        self.MoveSolutions.sort(key = lambda solution: solution.TotalTime, reverse=False) # sort solutions ascending by TotalTime
        self.MoveSolutions.sort(key = lambda solution: solution.TotalScore, reverse=True) # sort solutions descending by TotalScore

        if len(self.MoveSolutions) == 0:
            print("There was no feasible move in the Neighborhood.")
            return None
            
        else:
            bestNeighborhoodSolution = self.MoveSolutions[0]
        return bestNeighborhoodSolution
        

    def Update(self, sequence):
        self.Sequence = sequence

        self.Moves.clear()
        self.MoveSolutions.clear()

    def LocalSearch(self, neighborhoodEvaluationStrategy, solution):

        hasSolutionImproved = True

        while hasSolutionImproved:
            self.Update(solution.Sequence)
            self.DiscoverMoves()
            self.EvaluateMoves(neighborhoodEvaluationStrategy)
        
            #Lösung beibehalten, wenn kein besserer Move möglich war, sonst ersetzen
            if self.MakeBestMove() == None:
                bestNeighborhoodSolution = solution
            else:
                bestNeighborhoodSolution = self.MakeBestMove()

            #wenn die Lösung besser war, einmal alles updaten
            if bestNeighborhoodSolution.TotalScore > solution.TotalScore:
                print("New best solution has been found!")
                print(bestNeighborhoodSolution)

                self.SolutionPool.AddSolution(bestNeighborhoodSolution)

                solution.Sequence = bestNeighborhoodSolution.Sequence
                solution.TotalScore = bestNeighborhoodSolution.TotalScore
                solution.TotalTime = bestNeighborhoodSolution.TotalTime
            
            #war nur die Zeit besser, ist das auch einen nennenswerte Verbesserung
            elif (bestNeighborhoodSolution.TotalScore == solution.TotalScore) and (bestNeighborhoodSolution.TotalTime < solution.TotalTime):
                print(f"Time improvement made: {bestNeighborhoodSolution.TotalTime}")
            
                self.SolutionPool.AddSolution(bestNeighborhoodSolution)

                solution.Sequence = bestNeighborhoodSolution.Sequence
                solution.TotalScore = bestNeighborhoodSolution.TotalScore
                solution.TotalTime = bestNeighborhoodSolution.TotalTime
            
            #Abbruch, wenn keine Verbesserung möglich war
            else:
                print(f"Reached local optimum of {self.Type} neighborhood. Stop local search.")
                hasSolutionImproved = False

""" Represents the swap of the element at IndexA with the element at IndexB for a given sequence (= solution). """
class SwapMove: # nur Abwandlug aus VL
    def __init__(self, initialSequence, indexA, indexB):
        self.Sequence = list(initialSequence) # create a copy of the sequence
        self.IndexA = indexA
        self.IndexB = indexB

        self.Sequence[indexA] = initialSequence[indexB]
        self.Sequence[indexB] = initialSequence[indexA]
        
""" Contains all $n choose 2$ swap moves for a given Sequence (= solution). """
class SwapNeighborhood(BaseNeighborhood): # nur Abwandlung aus VL
    def __init__(self, inputData, initialSequence, evaluationLogic, solutionPool):
        super().__init__(inputData, initialSequence, evaluationLogic, solutionPool)

        self.Type = 'Swap'

    """ Generate all $n choose 2$ moves. """
    def DiscoverMoves(self):
        for i in range(len(self.Sequence)):
            for j in range(len(self.Sequence)):
                if i < j:
                    swapMove = SwapMove(self.Sequence, i, j)
                    self.Moves.append(swapMove)

""" Represents the insertion of the element at IndexA at the new position IndexB for a given sequence (= solution). """
class InsertionMove: # nur Abwandlung aus VL
    def __init__(self, initialSequence, indexA, indexB):
        self.Sequence = [] # create a copy of the sequence
        self.IndexA = indexA
        self.IndexB = indexB

        for k in range(len(initialSequence)):
            if k == indexA:
                continue

            self.Sequence.append(initialSequence[k])

        self.Sequence.insert(indexB, initialSequence[indexA])

""" Contains all $(n - 1)^2$ insertion moves for a given sequence (= solution). """
class InsertionNeighborhood(BaseNeighborhood): # nur Abwandlung aus VL
    def __init__(self, inputData, initialSequence, evaluationLogic, solutionPool):
        super().__init__(inputData, initialSequence, evaluationLogic, solutionPool)

        self.Type = 'Insertion'

    def DiscoverMoves(self):
        for i in range(len(self.Sequence)):
            for j in range(len(self.Sequence)):
                if i == j or i == j + 1:
                    continue

                insertionMove = InsertionMove(self.Sequence, i, j)
                self.Moves.append(insertionMove)

class AddMove: #Move der schaut, ob noch irgendwo(!) ein weiterer Node eingefügt werden kann
    def __init__(self, initialSequence, index, element ):
        self.Sequence = list(initialSequence)
        self.Sequence.insert(index, element)
        
class AddNeighborhood(BaseNeighborhood):
    def __init__(self, inputData, initialSequence, evaluationLogic, solutionPool):
        super().__init__(inputData, initialSequence, evaluationLogic, solutionPool)

        self.Type = "Add"
        self.Sequence = initialSequence
        #"Residue" sind die nicht erreichten Knoten
        self.Residue = [x for x in range(2,self.InputData.NodeCount+1) if x not in self.Sequence]

    def DiscoverMoves(self):
        for node in self.Residue:
            for position in range(len(self.Sequence)+1):
                addMove = AddMove(self.Sequence, position, node)
                self.Moves.append(addMove)

#ReMove wird aktuell nicht genutzt
class ReMove: #entfernt einfach einen Knoten, Idee dahinter war Dekonstruktion für Rekonstruktion, funktioniert aber nicht gut mit VNS
    def __init__(self, initialSequence, index):
        self.Sequence = list(initialSequence)
        self.Sequence.pop(index)

class ReNeighborhood(BaseNeighborhood):
    def __init__(self, inputData, initialSequence, evaluationLogic, solutionPool):
        super().__init__(inputData, initialSequence, evaluationLogic, solutionPool)

        self.Type = "Re"
        self.Sequence = initialSequence

    def DiscoverMoves(self):
        for index in range(len(self.Sequence)):
            reMove = ReMove(self.Sequence, index)
            self.Moves.append(reMove)

class ExchangeMove: #Tauscht erreichten knoten mit einem Knoten aus der Menge aus nicht erreichten Knoten
    def __init__(self, initialSequence, index, element):
        self.Sequence = list(initialSequence)
        self.Sequence[index] = element
class ExchangeNeighborhood(BaseNeighborhood):
    def __init__(self, inputData, initialSequence, evaluationLogic, solutionPool):
        super().__init__(inputData, initialSequence, evaluationLogic, solutionPool)

        self.Type = "Exchange"
        self.Sequence = initialSequence
        #"Residue" sind die nicht erreichten Knoten
        self.Residue = [x for x in range(2,self.InputData.NodeCount+1) if x not in self.Sequence]

    def DiscoverMoves(self):
        for node in self.Residue:
            for index in range(len(self.Sequence)):
                exchangeMove = ExchangeMove(self.Sequence, index, node)
                self.Moves.append(exchangeMove)

class BlockExchangeMove: # entfernt einen 2er Block aus der Lösung und füllt Lücke mit unerreichten Knoten
    def __init__(self, initialSequence, index, element1, element2):
        self.Sequence = list(initialSequence)
        #Substituieren der Elemente in der Sequenz
        self.Sequence[index:index+1] = [element1, element2]
        
class BlockExchangeNeighborhood(BaseNeighborhood):
    def __init__(self, inputData, initialSequence, evaluationLogic, solutionPool):
        super().__init__(inputData, initialSequence, evaluationLogic, solutionPool)

        self.Type = "BlockExchange"
        self.Sequence = initialSequence
        #"Residue" sind die nicht erreichten Knoten
        self.Residue = [x for x in range(2,self.InputData.NodeCount+1) if x not in self.Sequence]

    def DiscoverMoves(self):
        for element1 in self.Residue:
            for element2 in self.Residue:
                if element1 != element2:
                    for index in range(len(self.Sequence)):
                        blockExchangeMove = BlockExchangeMove(self.Sequence, index, element1, element2)
                        self.Moves.append(blockExchangeMove)

class ExchangeMove: #Tauscht erreichten knoten mit einem Knoten aus der Menge aus nicht erreichten Knoten
    def __init__(self, initialSequence, index, element):
        self.Sequence = list(initialSequence)
        self.Sequence[index] = element
class ExchangeNeighborhood(BaseNeighborhood):
    def __init__(self, inputData, initialSequence, evaluationLogic, solutionPool):
        super().__init__(inputData, initialSequence, evaluationLogic, solutionPool)

        self.Type = "Exchange"
        self.Sequence = initialSequence
        #"Residue" sind die nicht erreichten Knoten
        self.Residue = [x for x in range(2,self.InputData.NodeCount+1) if x not in self.Sequence]

    def DiscoverMoves(self):
        for node in self.Residue:
            for index in range(len(self.Sequence)):
                exchangeMove = ExchangeMove(self.Sequence, index, node)
                self.Moves.append(exchangeMove)

---
### Solver
---

In [7]:
import numpy
import os

In [8]:
class Solver: # analog zur Vorlesung bis auf Write Methode
    def __init__(self, inputData, seed):
        self.InputData = inputData
        self.Seed = seed
        self.RNG = numpy.random.default_rng(seed)

        self.EvaluationLogic = EvaluationLogic(inputData)
        self.SolutionPool = SolutionPool()
        
        self.ConstructiveHeuristic = ConstructiveHeuristics(self.EvaluationLogic, self.SolutionPool)

    def ConstructionPhase(self, constructiveSolutionMethod):
        self.ConstructiveHeuristic.Run(self.InputData, constructiveSolutionMethod)

        bestInitalSolution = self.SolutionPool.GetHighestTotalScoreSolution()

        print("Constructive solution found.")
        print(bestInitalSolution)
        return bestInitalSolution
    
    def ImprovementPhase(self, startSolution, algorithm):
        algorithm.Initialize(self.EvaluationLogic, self.SolutionPool, self.RNG)
        bestSolution = algorithm.Run(startSolution)

        print("Best found Solution.")
        print(bestSolution)

    def RunLocalSearch(self, constructiveSolutionMethod, algorithm):
        startSolution = self.ConstructionPhase(constructiveSolutionMethod)

        self.ImprovementPhase(startSolution, algorithm)
    
    def WriteSolution(self): #Erstellt Solution Ordner und schreibt Lösungsdatei
        solution = self.SolutionPool.GetHighestTotalScoreSolution()
        #Erstellen des Dicts mit der Lösung
        result = {"TotalScore":solution.TotalScore,
                  "Permutation": str([1]+solution.Sequence+[1]),
                  "Duration":solution.TotalTime}
        #Erstellen des Ordners
        os.makedirs("Solutions", exist_ok=True)
        #derive path
        file_Path = os.path.abspath(self.InputData.Path)
        #Speicherpath erstellen
        file_Name = os.path.join("Solutions", "Solution-" + os.path.basename(file_Path))
        #File abspeichern
        with open(file_Name, 'w') as file:
            json.dump(result, file, indent=4)

---
### Improvement Algorithm
---

In [9]:
from copy import deepcopy
import time
import signal

In [10]:
""" Base class for several types of improvement algorithms. """ 
class ImprovementAlgorithm: #analog zur Vorlesung
    def __init__(self, inputData, neighborhoodEvaluationStrategy = 'BestImprovement', neighborhoodTypes = ['Swap']):
        self.InputData = inputData

        self.EvaluationLogic = {}
        self.SolutionPool = {}
        self.RNG = {}

        self.NeighborhoodEvaluationStrategy = neighborhoodEvaluationStrategy
        self.NeighborhoodTypes = neighborhoodTypes
        self.Neighborhoods = {}

    def Initialize(self, evaluationLogic, solutionPool, rng = None):
        self.EvaluationLogic = evaluationLogic
        self.SolutionPool = solutionPool
        self.RNG = rng

    """ Similar to the so-called factory concept in software design. """
    def CreateNeighborhood(self, neighborhoodType, bestCurrentSolution):
        if neighborhoodType == 'Swap':
            return SwapNeighborhood(self.InputData, bestCurrentSolution.Sequence, self.EvaluationLogic, self.SolutionPool)
        elif neighborhoodType == 'Insertion':
            return InsertionNeighborhood(self.InputData, bestCurrentSolution.Sequence, self.EvaluationLogic, self.SolutionPool)
        else:
            raise Exception(f"Neighborhood type {neighborhoodType} not defined.")

    def InitializeNeighborhoods(self, solution):
        for neighborhoodType in self.NeighborhoodTypes:
            neighborhood = self.CreateNeighborhood(neighborhoodType, solution)
            self.Neighborhoods[neighborhoodType] = neighborhood

""" Iterative improvement algorithm through sequential variable neighborhood descent. """
class IterativeImprovement(ImprovementAlgorithm): #analog zur Vorlesung
    def __init__(self, inputData, neighborhoodEvaluationStrategy = 'BestImprovement', neighborhoodTypes = ['Swap']):
        super().__init__(inputData, neighborhoodEvaluationStrategy, neighborhoodTypes)

    def Run(self, solution):
        self.InitializeNeighborhoods(solution)    

        # According to "Hansen et al. (2017): Variable neighorhood search", this is equivalent to the 
        # sequential variable neighborhood descent with a pipe neighborhood change step.
        for neighborhoodType in self.NeighborhoodTypes:
            neighborhood = self.Neighborhoods[neighborhoodType]

            neighborhood.LocalSearch(self.NeighborhoodEvaluationStrategy, solution)
        
        return solution

#Implementierung der Metaheuristic als Erweiterung der Basisklasse ImprovementAlgorithm
class VariableNeighborhoodSearch(ImprovementAlgorithm):
    def __init__(self, inputData, acceptanceCriterion, neighborhoodTypes = ["Add","Exchange","BlockExchange"]):
        super().__init__(inputData)
        #Für die Lokale Suche wird VND aus der VL genutzt
        self.LocalSearchAlgorithm = IterativeImprovement(self.InputData, neighborhoodTypes = ["Insertion","Swap"])
        self.AcceptanceCriterion = acceptanceCriterion
        self.NeighborhoodTypes = neighborhoodTypes
    
    def Initialize(self, evaluationLogic, solutionPool, rng = numpy.random.default_rng(161)):
        self.EvaluationLogic = evaluationLogic
        self.SolutionPool = solutionPool
        self.RNG = rng

        self.LocalSearchAlgorithm.Initialize(self.EvaluationLogic, self.SolutionPool)# weil "externe" Local Search Methdode
    
    def Run(self, currentSolution):
        runTime = time.time()
        currentSolution = deepcopy(currentSolution)

        ###Acceptance Criterion
        def handler(signum, frame):
            print("Aktuell beste Lösung:")
            print(self.SolutionPool.GetHighestTotalScoreSolution())
            raise TimeoutError("Maximum Runtime reached.")
        # Set the alarm signal handler
        signal.signal(signal.SIGALRM, handler)
        # Set the alarm to go off after the specified runtime
        signal.alarm(self.AcceptanceCriterion)

        k = 0
        while k <= (len(self.NeighborhoodTypes)-1):
            ###Shake -> erstellen einer Nachbarschaft + random Auswahl einer Lösung
            currentNeighborhoodType = self.NeighborhoodTypes[k]
            currentNeighborhood = None
            if currentNeighborhoodType == "Insertion":
                currentNeighborhood = InsertionNeighborhood(self.InputData, currentSolution.Sequence, self.EvaluationLogic, self.SolutionPool)
            elif currentNeighborhoodType == "Swap":
                currentNeighborhood = SwapNeighborhood(self.InputData, currentSolution.Sequence, self.EvaluationLogic, self.SolutionPool)
            elif currentNeighborhoodType == "Add":
                currentNeighborhood = AddNeighborhood(self.InputData, currentSolution.Sequence, self.EvaluationLogic, self.SolutionPool)
            elif currentNeighborhoodType == "Exchange":
                currentNeighborhood = ExchangeNeighborhood(self.InputData, currentSolution.Sequence, self.EvaluationLogic, self.SolutionPool)
            elif currentNeighborhoodType == "Re":
                currentNeighborhood = ReNeighborhood(self.InputData, currentSolution.Sequence, self.EvaluationLogic, self.SolutionPool)
            elif currentNeighborhoodType == "BlockExchange":
                currentNeighborhood = BlockExchangeNeighborhood(self.InputData, currentSolution.Sequence, self.EvaluationLogic, self.SolutionPool)
            else:
                raise Exception("Neighborhood type not defined.")
            currentNeighborhood.Update(currentSolution.Sequence)
            currentNeighborhood.DiscoverMoves()
            currentNeighborhood.EvaluateMoves(self.NeighborhoodEvaluationStrategy)
            #Checken, ob Nachbarschaft leer ist
            if len(currentNeighborhood.MoveSolutions) > 0:
                #Auswahl einer zufälligen Lösung
                currentSolution = self.RNG.choice(currentNeighborhood.MoveSolutions)
                print("Shake:", currentSolution)

            ###Local Search -> Verbesserung der Lösung durch Intra Tour Tausche mit VND und Swap + Insertion Nachbarschaft
                currentSolution = self.LocalSearchAlgorithm.Run(currentSolution)
            ###Shake
            else:
                print("Unable to shake due to empty Neighborhood")
            ###Change Neighborhood
            #Wenn Verbesserung erzielt werden Nachbarschaften von vorne angefangen, sonst skip auf nächste Nachbarschaft
            if currentSolution.TotalScore > self.SolutionPool.GetHighestTotalScoreSolution().TotalScore:# war vorher totaltime
                self.SolutionPool.AddSolution(currentSolution)
                k = 0
                print(f"New best Solution found, resetting k to {k}.")
            else:
                k += 1
                print(f"No better solution found, incrementing k to {k}.")
        print(f"RunTime: {time.time() - runTime} s")   
        return self.SolutionPool.GetHighestTotalScoreSolution()

---
### Results
---

In [11]:
data = InputData("Testinstanzen/Instance_4.json")
vns = VariableNeighborhoodSearch(data, 180)
#vns.NeighborhoodEvaluationStrategy = "FirstImprovement"

solver = Solver(data, 2048)
#print(solver.ConstructionPhase("HSF"))
solver.RunLocalSearch(constructiveSolutionMethod = "NNF", algorithm = vns)
solver.WriteSolution()

Generating an initial solution according to NNF.
Constructive solution found.
Sequence [1, 63, 6, 49, 90, 10, 84, 72, 21, 74, 59, 17, 15, 11, 32, 91, 98, 23, 45, 47, 93, 28, 67, 58, 61, 25, 81, 69, 73, 50, 44, 2, 54, 40, 64, 68, 85, 39, 1] collects 1949 score points in 10345.062827513684 time units
Shake: Sequence [1, 63, 6, 49, 90, 10, 84, 72, 21, 74, 59, 17, 15, 11, 32, 91, 98, 23, 45, 47, 93, 28, 67, 58, 61, 25, 81, 69, 73, 50, 44, 2, 54, 40, 64, 68, 85, 39, 87, 1] collects 2049 score points in 10504.490697411231 time units
Time improvement made: 10397.751843545362
Time improvement made: 10357.932287649504
Time improvement made: 10345.195091353708
Time improvement made: 10301.055931318459
Time improvement made: 10215.27247500328
Time improvement made: 10206.13089307518
Reached local optimum of Insertion neighborhood. Stop local search.
Reached local optimum of Swap neighborhood. Stop local search.
No better solution found, incrementing k to 1.
Shake: Sequence [1, 63, 6, 49, 90, 10, 

: 